Drone flying up

In [4]:
using Dojo
using DojoEnvironments
using LinearAlgebra

# ### Environment
quadrotor_env = get_environment(:quadrotor_waypoint; horizon=2000)

# ### PID Controller Parameters
Kp = [2.0, 2.0, 2.0]   # Proportional gain
Ki = [0.1, 0.1, 0.1]   # Integral gain
Kd = [0.1, 0.1, 0.1]   # Derivative gain

# State for the integral term and previous error
integral_error = zeros(3)
previous_error = zeros(3)
# Function to distribute thrust to the four rotors
function distribute_thrust(thrust)
    return [thrust[3], thrust[3], thrust[3], thrust[3]]
end
# Velocity controller
function velocity_controller!(environment, v_des, ω_des, dt)
    state = get_state(environment)
    linear_velocity = state[7:9] # vx, vy, vz

    # Error terms
    error = v_des .- linear_velocity
    thrust = (10 .* error .- 1 .* linear_velocity .+ 5.1) # P, D, feedforward

    # Debugging prints
    println("Velocity Error: ", error)
    println("Thrust: ", thrust)

    rpm_thrust = distribute_thrust(thrust)  # Map thrust to 4 rotors
    rpm_torque = attitude_controller!(environment, ω_des, dt)  # Attitude control

    # Combine thrust and torque (simplified)
    rpm = rpm_thrust .* 20 .* trans_mode .+ rpm_torque
    println("RPM: ", rpm)
    set_input!(environment, rpm)
end
# Position controller
function position_controller!(environment, pos_des, dt)
    state = get_state(environment)
    pos_is = state[1:3]
    v_des = pos_des .- pos_is

    # Desired angular velocities (tune these based on the desired orientation)
    ω_des = [0.0, 0.0, 0.0]

    # Debugging prints
    println("Position Desired: ", pos_des)
    println("Position Error: ", v_des)

    velocity_controller!(environment, v_des, ω_des, dt)
end
# PID Attitude controller
function attitude_controller!(environment, ω_des, dt)
    global integral_error
    global previous_error

    state = get_state(environment)
    angular_velocity = state[10:12] # ωx, ωy, ωz 

    # Error terms
    error = ω_des .- angular_velocity
    integral_error += error * dt
    derivative_error = (error - previous_error) / dt

    torque = Kp .* error .+ Ki .* integral_error .+ Kd .* derivative_error
    previous_error = error

    # Debugging prints
    println("Attitude Error: ", error)
    println("Integral Error: ", integral_error)
    println("Derivative Error: ", derivative_error)
    println("Torque: ", torque)

    # Apply the torque to the motors
    rpm_torque = [torque[1], -torque[1], torque[2], -torque[2]]

    return rpm_torque
end
# Define waypoints
waypoints = [[0.0, 0.0, 0.5], [1.0, 0.0, 0.5], [1.0, 1.0, 0.5], [0.0, 1.0, 0.5], [0.0, 0.0, 0.5]]
current_waypoint_index = 1

# Main controller
function controller!(environment, k)
    global current_waypoint_index

    dt = 0.1 # Time step (example value, should be based on actual simulation time step)

    pos_des = waypoints[current_waypoint_index]  # Current target position
    position_controller!(environment, pos_des, dt)

    # Move to the next waypoint if close enough to the current one
    current_pos = get_state(environment)[1:3]
    if norm(current_pos .- pos_des) < 0.1
        current_waypoint_index = current_waypoint_index % length(waypoints) + 1
    end

    # Debugging prints
    println("Current Position: ", current_pos)
    println("Current Waypoint: ", pos_des)
    println("Waypoint Index: ", current_waypoint_index)
end
# ### Simulate
initialize!(quadrotor_env, :quadrotor)
simulate!(quadrotor_env, controller!; record=true)

# ### Visualize
vis = visualize(quadrotor_env)
render(vis)


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("Background" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("top_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64], "bottom_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Grid" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("visible" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x70, 0x61, 0x74, 0x68, 0xa5, 0x2f, 0x47, 0x72, 0x69, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoints" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoint1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "floor" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("bodies" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rotor_3__id_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x33, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_1__id_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x31, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "base_link__id_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6c, 0x69, 0x6e, 0x6b, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_0__id_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 